# Dispersal simulations

In this example we will run a simulation of the spread of the Spotted Winged
Drosophila _D. suzukii_ accross the continental USA.

## Setup

First, load the required packages. Dates is a core julia package that
give us date/time handling, GeoData simplifies the loading of geospatial
raster files. It requires also loading NCDatasets.jl and ArchGDAL.jl to load
NetCDF and GeoTiff files, respectively.

In [ ]:
# using Pkg
# Pkg.add(PackageSpec(url="https://github.com/rafaqz/GeoData.jl", rev="master"))
using GeoData, DimensionalData, Dispersal, Dates, NCDatasets, ArchGDAL, Plots
using DimensionalData: rebuild, Forward, Reverse
basedir = joinpath(dirname(Base.pathof(Dispersal)), "../docs")

### Define simulation settings

We use DateTime units of for the timestep:

In [ ]:
timestep = Month(1);
tspan = DateTime(2016, 1), DateTime(2018, 12)
nframes = length(tspan[1]:timestep:tspan[2])
aust = Lon(Between(113.338953078, 153.569469029)),
       Lat(Between(-43.6345972634, -10.6681857235));

---

### Define a `RuleSet`: climate driven population growth

This will involve combining multiple dispersal componenents into a single
`RuleSet` object: population growth, local dispersal, Allee effects, and human
dispersal.

Follow the [examples tutorial](https://rafaqz.github.io/GrowthRates.jl/dev/example/index.html) over
at [GrowthRates.jl](https://github.com/rafaqz/GrowthRates.jl). To skip this step
just download the output saved in the example:

In [ ]:
dataurl = "https://media.githubusercontent.com/media/cesaraustralia/Dispersal.jl/data"
growthratesfilename = "growthrates.ncd"
growthratesfilepath = joinpath(basedir, growthratesfilename)
isfile(growthratesfilepath) ||
  download(joinpath(dataurl, growthratesfilename), joinpath(basedir, growthratesfilename));

And select just the bounding box for Australia, and set the dimension order to Lat/Lon:

In [ ]:
growthrates = NCDarray(growthratesfilepath)[aust...] |>
    A -> permutedims(A, (Lat, Lon, Ti)) |>
    A -> reorderarray(A, (Lat(Reverse()), Lon(Forward())));

Set the length of the timestep. TODO: get this automatically from NetCDF units in GeoData.jl

In [ ]:
ti = dims(growthrates, Ti)
mode = Sampled(order(ti), Regular(timestep), Intervals(Start()))
growthrates = setdims(growthrates, rebuild(ti; mode=mode));

Plot the growth layer:

In [ ]:
growthrates[Ti(1)] |> plot
savefig(joinpath(basedir, "build/assets/growthrates.png"));

Create a `ExactLogisticGrowthMap` rule from the layer, here we use
unitful units for the layers' time dimension:

In [ ]:
carrycap = 1e9
growth = ExactLogisticGrowthMap(layer=growthrates, carrycap=carrycap);

### Local dispersal

In [ ]:
λ = 0.0125
radius = 1
sze = 2radius + 1
dm = AreaToArea(30)
@time hood = DispersalKernel{radius}(;kernel=zeros(Float64, radius, radius),
                               formulation=ExponentialKernel(λ), distancemethod=dm)
localdisp = InwardsPopulationDispersal(hood);

### Allee effects

In [ ]:
allee = AlleeExtinction(minfounders=10.0);

### Human dispersal

The human dispersal component is generated from an array or population data.
First we'll open an input tiff, and move it to a memory backed `GeoArray`.

In [ ]:
humanpop_filename = "population_density.tif"
humanpop_filepath = joinpath(basedir, humanpop_filename)
isfile(humanpop_filepath) || download(joinpath(dataurl, humanpop_filename), humanpop_filepath);

Again select the Australian bounds. This time we also select the first Band.
Tiff data always has bands, even when there is only one:

In [ ]:
humanpop = GDALarray(humanpop_filepath; usercrs=EPSG(4326))[Band(1), aust...] |>
    A -> permutedims(A, (Lat, Lon)) |> reorderarray
savefig(joinpath(basedir, "build/assets/humanpop.png"));

![popgrowth](assets/humanpop.png)

In [ ]:
@time humandisp = HumanDispersal(
    human_pop=parent(humanpop),
    human_exponent=2.0,
    dist_exponent=2.0,
    dispersalperpop=1e-9,
    max_dispersers=500.0,
    nshortlisted=50,
    scale=8,
);

### Define initialisation data

Make a zeros array and populate the starting cells. We need to replace the `missing`
values with something else Dispersal.jl init can't contain `missing` or it will spread everywhere:

In [ ]:
init = replace_missing(growthrates[Ti(1)], NaN) |> zero

incursion = [(36.9677,-122.0294),
             (37.3226,-121.8921),
             (37.6008,-120.9545),
             (35.3453,-119.0586),
             (38.7318,-121.9014),
             (38.5249,-121.9708),
             (37.2502,-119.751)]

# Using `Contains` finds the interval that contains the coordinates
for (lat, lon) in incursion
    init[Lat(Near(lat)), Lon(Near(lon))] = 1e7
end;

### Define a masking layer

This layer lets the simulation know which cells should be ignored.

In [ ]:
masklayer = boolmask(growthrates[Ti(1)])
plot(masklayer)
savefig(joinpath(basedir, "build/assets/mask.png"));

![mask](assets/mask.png)

### Define a combined ruleset

In [ ]:
ruleset = Ruleset(
    humandisp, Chain(localdisp, allee, growth);
    init=init,
    mask=masklayer,
    timestep=timestep,
    minval=0.0,
    maxval=carrycap,
);

---

## Output

The simplest and oftern the most performant output for a simulation is an
ArrayOutput, which simply writes the simulation to a preallocated array
without visualising it.

In [ ]:
output = ArrayOutput(init, nframes);

### Run a simulation

To run a simulation, we pass in the output and rulset.

In [ ]:
sim!(output, ruleset; tspan=tspan);

### Save a gif of your simulation

Gif files are an easy way to share the visual dynamics of the simulation. First
we need to define a color processor to turn the simulation into images for the
frames of the the gif. This processor can also be used later in web or gtk
outputs.

You can use the built-in `Greyscale` scheme, or any scheme from
[ColorSchemes.jl](https://github.com/JuliaGraphics/ColorSchemes.jl).

In [ ]:
using ColorSchemes, Colors
scheme = ColorSchemes.Oranges_3
#scheme = ColorSchemes.autumn1

## Frame Processing Colors
zerocolor = RGB24(0.7)
maskcolor = RGB24(0.0)
textconfig = TextConfig(font="arial")
processor = ColorProcessor(scheme, zerocolor, maskcolor, textconfig);

With a non-image output like ArrayOuput we need to pass in the image processor
manually.

In [ ]:
savegif(joinpath(basedir, "build/assets/sim.gif"), output; minval=0.0, maxval=1e7, processor=processor, fps=10);

![Drosphila suzukii spread](assets/sim.gif)


## Live simulation outputs

There are a number of live outputs provided in CelularAutomataBase.jl and
specific output packages DynamicGridsGtk and DynamicGridsInteract that keep
heavy graphics and web dependencies separate form the rest of the framework.


### REPL output

You can view a simulation over SSH or just in your local console using the
included `REPLOutput`. It doesn't work so well in Atom/Juno, so we only recommend
using it in a real terminal. It also works better in a terminal where you can
easily reduce the font size.

In [ ]:
using Crayons
output = REPLOutput(init; style=Block(), fps=5, color=:white, store=false)
sim!(output, ruleset; tspan=tspan);

The `Braile()` style is half the dimensions of the `Block()` style, but doen't
look as clear.

In [ ]:
output = REPLOutput(init; style=Braile(), fps=5, color=:white, store=false)
sim!(output, ruleset; tspan=tspan);

### Interactive web outputs

[DynamicGridsInteract.jl](http://gihub.com/rafaqz/DynamicGridsInteract.jl)
produces interactive web page outputs for a simulation. It uses Interact.jl for
live control, providing a control console and sliders for model parameters, even
for your own custom models.

The simple `InteractOutput()` is the core output that can run on its own inside Juno
or a Jupyter notebook. It can also be served to a browser locally or over the
web using `ServerOutput()`, or run in a standalone desktop app using
`ElectronOutput()`.

### Juno or jupyter notebooks

Building a `InteractOutput()` and running `display()` will open an output in a plot
pane in Juno. See the example above to define an image processor. Setting
`store` to true will save the last simulation to the output array to be saved or
converted to a gif. Really long simulations may use your avilable ram, in which
case set `store` to false.

In [ ]:
using DynamicGridsInteract
output = InteractOutput(init, ruleset;
    tspan=tspan,
    fps=10,
    store=true,
    processor=processor,
    slider_throttle=1.0,
)
display(output)

### Wrap the IneractOutput in a standalone desktop (electron) app

This will create a standalone desktop app wrapping the InteractOutput. Unfortunately
the compile and load time for electron can take quite a while.

In [ ]:
output = ElectronOutput(init, ruleset;
    tspan=tspan,
    fps=10,
    store=true,
    processor=processor,
    slider_throttle=1.0,
)

### Serving web pages

The `InteractOutput` can be served locally or over the web. You may need to do some
port or firewall configuration on your server, but otherwise this is all you
need to do to serve a simulation. Unlike other outputs, `ServerOutput` makes a
copy of the internal `InteractOutput` for each new connection. Changes in one
connection will not effect anything in the others.

In [ ]:
output = ServerOutput(init, ruleset;
    tspan=tspan,
    fps=10,
    processor=processor,
    slider_throttle=1.0,
    port=8080,
)

If run locally, the interface should be served at the address "localhost:8080" in your web browser.

### Run in a GTK window

The `GTKOutput` in
[DynamicGridsGtk.jl](http://gihub.com/rafaqz/DynamicGridsGtk.jl) is a
simple desktop output that just shows the simulation without controls. It can be
useful for faster load-time than `ElectronOutput`, and also dedicates all
screenspace to viewing the simulation.

In [ ]:
using DynamicGridsGtk
output = GtkOutput(init .* 0;
    fps=10,
    store=true,
    processor=processor
)
sim!(output, ruleset; tspan=tspan)

---

## Adding your own rules

In Dispersal.jl and the CelularAutomataBase.jl framework it's easy to add your
won custom model components, or 'rules' to follow the language of cellular
automata. If you write them well they will perform as well as the built in
rules, and can take advantage of a number of the same performance optimisations
by using the type heirarchy in DynamicGrids.jl.

Rules may be a [number of types].